In [12]:
import gymnasium as gym
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from collections import deque
import random

In [22]:
# Define the DQN agent class
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # Discount factor
        self.epsilon = 0.5  # Exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(optimizer=Adam(), loss=MeanSquaredError())
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.randint(self.action_size)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def replay(self, batch_size):
        minibatch = np.array(random.sample(self.memory, batch_size))
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                p = self.model.predict(next_state)
                print(p, reward, self.gamma, np.amax(p[0]))
                target = reward + self.gamma * np.amax(p[0])
                print(f"Target: {target}")
            target_f = self.model.predict(state)
            print(f"Target_f: {target_f}")
            target_f[0][action] = target
            print(f"Target_f f: {target_f}")
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [23]:
# Create the environment
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

# Initialize the DQN agent
agent = DQNAgent(state_size, action_size)

In [24]:
# Create the environment
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

# Initialize the DQN agent
agent = DQNAgent(state_size, action_size)

# Training loop
batch_size = 32
num_episodes = 1000
for episode in range(num_episodes):
    state, _ = env.reset()
    state = np.reshape(state, [1, state_size])
    for t in range(500):
        # Render the environment (optional)
        env.render()

        # Choose an action
        action = agent.act(state)

        # Perform the action
        next_state, reward, done, _, _ = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])

        # Remember the experience
        agent.remember(state, action, reward, next_state, done)

        # Update the state
        state = next_state

        # Check if episode is finished
        if done:
            break

        # Train the agent
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gymnasium/envs/classic_control/cartpole.py:215: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(


1/1 [==============================] - 0s 26ms/step
[[-0.00768261  0.09743714]] 1.0 0.95 0.097437136
Target: 1.0925652790814637


/var/folders/61/wf0bbt912gb82tgxbnlz82bm0000gn/T/ipykernel_7873/1508857333.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  minibatch = np.array(random.sample(self.memory, batch_size))


1/1 [==============================] - 0s 41ms/step
Target_f: [[-0.02687504  0.18128891]]
Target_f f: [[-0.02687504  1.0925653 ]]
1/1 [==============================] - 0s 54ms/step
[[ 0.02684633 -0.0489723 ]] 1.0 0.95 0.026846334
Target: 1.0255040176212789


KeyboardInterrupt: 

In [25]:
# create a vector of shape (3, 10)
v = np.random.rand(3, 10)
v

array([[0.17453177, 0.16446972, 0.3238309 , 0.92902398, 0.10876505,
        0.81238998, 0.85164243, 0.24084329, 0.83042923, 0.20552261],
       [0.87247842, 0.00919696, 0.46001422, 0.65592522, 0.01076522,
        0.61507032, 0.39647739, 0.88890132, 0.75641648, 0.93754863],
       [0.56118765, 0.98328904, 0.35222312, 0.25960215, 0.38626687,
        0.8689649 , 0.07503603, 0.54959071, 0.42807817, 0.84962261]])

# Swarm

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

class QLearningAgent:
    def __init__(self, state_size, action_size, learning_rate=0.001):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = []
        self.gamma = 0.95    # discount rate
        self.epsilon = 0.5   # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = learning_rate
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='softmax'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        
        if np.random.rand() <= self.epsilon:
            action = np.random.choice(self.action_size, len(state))
        else:
            action = np.argmax(self.model.predict(state, verbose=0), axis=1)
        
        # Translate discretize action to environment action
        act_values = []
        for a in action:
            if a == PICK_UP:
                act_values.append({
                    "action": environment.PICK_UP,
                    "move": [0, 0]
                })
            elif a == DROP:
                act_values.append({
                    "action": environment.PUT_DOWN,
                    "move": [0, 0]
                })
            elif a == S:
                act_values.append({
                    "action": environment.MOVE,
                    "move": [1, 0]
                })
            elif a == SE:
                act_values.append({
                    "action": environment.MOVE,
                    "move": [1, 45]
                })
            elif a == E:
                act_values.append({
                    "action": environment.MOVE,
                    "move": [1, 90]
                })
            elif a == NE:
                act_values.append({
                    "action": environment.MOVE,
                    "move": [1, 135]
                })
            elif a == N:
                act_values.append({
                    "action": environment.MOVE,
                    "move": [1, 180]
                })
            elif a == NW:
                act_values.append({
                    "action": environment.MOVE,
                    "move": [1, 225]
                })
            elif a == W:
                act_values.append({
                    "action": environment.MOVE,
                    "move": [1, 270]
                })
            elif a == SW:
                act_values.append({
                    "action": environment.MOVE,
                    "move": [1, 315]
                })
                
        return act_values, action

    def replay(self, batch_size):
        minibatch = []
        
        # Sample a random batch from memory
        for _ in range(batch_size):
            minibatch.append(self.memory[np.random.randint(0, len(self.memory))])
        
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state, verbose=0), axis=1)
            
            target_f = self.model.predict(state, verbose=0)
            
            for i in range(len(action)):
                target_f[i][action[i]] = target[i]
            
            # target_f[0][action] = target
            # state shape 3 x 44, target_f shape 3 x 10
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay